## Structure des fichiers tsv

knownForTitles = tconst

- name.basics :         nconst / primaryName / birthYear / deathYear / primaryProfession / knownForTitles
- title.akas :          titleId / ordering / title / region / language / types / attributes / isOriginalTitle
- title.basics :        tconst / titleType / primaryTitle / originalTitle / isAdult / startYear / endYear / runtimeMinutes / genres
- title.principals :    tconst / ordering / nconst / category / job / characters
- title.ratings :       tconst / averageRating / numVotes


## Nombre de lignes par échantillon

Nous allons utiliser pour chaque dataframe un maximum de 1000 lignes afin d'avoir un échantillon représentatif sans avoir de latence sur le traitement des données.

Projet Intermovie
Ce Notebook a pour but d'analyser un dataset de films dans le but de récupérer plusieurs informations :

La liste des acteurs par film. >>>>>> DONE

La liste des films Américains (en gardant leur nom en français) et leur note moyenne. >>>>>> DONE

Les notes moyennes des différents genres.

La note moyenne de chaque acteur par rapport aux films dans lesquels il apparaît.

In [7]:
# Extension IPython rechargeant les modules avant que l'utilisateur saisisse du code.
%load_ext autoreload
%autoreload 2

# Import des librairies.
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
namebasics = pd.read_table("./data/name.basics.tsv")

In [ ]:
titleprincipals = pd.read_table("./data/title.principals.tsv")

In [14]:
titlebasics = pd.read_table("./data/title.basics.tsv")

In [10]:
titleakas = pd.read_table("./data/title.akas.tsv")

In [11]:
titleratings = pd.read_table('./data/title.ratings.tsv')

En dessous on nettoie le premier fichier (name.basics) nécéssaire pour trouver les acteurs par film

In [ ]:
namebasics = namebasics.dropna(axis=0)
namebasics = namebasics.drop_duplicates()
namebasics = namebasics.drop(['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis=1)


En dessous on nettoie le deuxième fichier (title.principals)

In [ ]:
titleprincipals = titleprincipals.dropna(axis=0)
titleprincipals = titleprincipals[["tconst","nconst","category"]][titleprincipals['category'].str.contains('actor|actress|self', regex=True)]

En dessous on nettoie le troisième fichier (title.basics)

In [ ]:
titlebasics = titlebasics.dropna(axis=0)
titlebasics = titlebasics.drop_duplicates()
titlebasics = titlebasics[["tconst","titleType","originalTitle"]][titlebasics['titleType'].str.contains('movie', regex=True)]

#Pour faire le tri uniquement sur les long métrage (movies) utiliser les ligne ssuivantes et commenter celle juste au dessus à la place
#titlebasics = titlebasics[["tconst","titleType","originalTitle"]][titlebasics['titleType'].str.contains('movie', regex=True)]
#titlebasics = titlebasics.drop(['titleType'])

On fait les merges nécéssaires pour obtenir le ficher qui nous permettra de faire la requète

In [ ]:
dfmergeone = namebasics.merge(titleprincipals,on='nconst',how='left')
dfmergetwo = dfmergeone.merge( titlebasics, on='tconst', how='left')
dfmergetwo.drop(columns=['category', 'titleType'], axis=1)
dfmergetwo = dfmergetwo.dropna(axis=0)
dfmergetwo = dfmergetwo.drop_duplicates()

On fait la requète pour trouver les acteurs par film

In [ ]:
df_final = dfmergetwo.groupby('originalTitle').agg({'primaryName': ','.join}, axis = 0)
df_final = df_final.to_csv('ArtistesParFilm.csv')

On fait une requète pour trouver les films par acteurs (j'avais juste envie)

In [ ]:
df_final2 = dfmergetwo.groupby('primaryName').agg({'originalTitle': ','.join}, axis = 0)
df_final2 = df_final2.to_csv('FilmsParArtistes.csv')

2EME REQUETE 

In [12]:
titleakas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0
3,tt0000001,4,Carmencita,US,\N,\N,\N,0
4,tt0000001,5,Carmencita,\N,\N,original,\N,1


In [15]:
titlebasics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [16]:
titleratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1550
1,tt0000002,6.1,186
2,tt0000003,6.5,1207
3,tt0000004,6.2,113
4,tt0000005,6.1,1934


In [34]:
titleakas = titleakas.dropna(axis=0)
titleakas = titleakas.drop_duplicates()
titleakas = titleakas[["titleId","region"]][titleakas['region'].str.contains('US', regex=True)]
titleakas = titleakas.rename(columns={'titleId': 'tconst'})

In [35]:
titleakas.head()

,tconst,region
3,tt0000001,US
10,tt0000002,US
23,tt0000005,US
34,tt0000006,US
37,tt0000007,US


In [22]:
titlebasics = titlebasics.dropna(axis=0)
titlebasics = titlebasics.drop_duplicates()
titlebasics = titlebasics.drop(['titleType', 'primaryTitle', 'isAdult', 'startYear','endYear','runtimeMinutes','genres'], axis=1)

#Pour faire le tri uniquement sur les long métrage (movies) utiliser les ligne ssuivantes et commenter celle juste au dessus à la place
#titlebasics = titlebasics[["tconst","titleType","originalTitle"]][titlebasics['titleType'].str.contains('movie', regex=True)]
#titlebasics = titlebasics.drop(['titleType'])

In [23]:
titlebasics.head()

,tconst,originalTitle
0,tt0000001,Carmencita
1,tt0000002,Le clown et ses chiens
2,tt0000003,Pauvre Pierrot
3,tt0000004,Un bon bock
4,tt0000005,Blacksmith Scene


In [24]:
titleratings = titleratings.dropna(axis=0)
titleratings = titleratings.drop_duplicates()
titleratings = titleratings.drop(['numVotes'], axis=1)

In [25]:
titleratings.head()

,tconst,averageRating
0,tt0000001,5.6
1,tt0000002,6.1
2,tt0000003,6.5
3,tt0000004,6.2
4,tt0000005,6.1


In [38]:
dfmergeone = titleakas.merge(titlebasics,on='tconst',how='left')
dfmergetwo = dfmergeone.merge( titleratings, on='tconst', how='left')
dfmergetwo = dfmergetwo.dropna(axis=0)
dfmergetwo = dfmergetwo.drop_duplicates()

In [42]:
df_final = dfmergetwo.to_csv('./data/titleFilmsUS.csv')

In [39]:
dfmergetwo.head()

,tconst,region,originalTitle,averageRating
0,tt0000001,US,Carmencita,5.6
1,tt0000002,US,Le clown et ses chiens,6.1
2,tt0000005,US,Blacksmith Scene,6.1
3,tt0000006,US,Chinese Opium Den,5.2
4,tt0000007,US,Corbett and Courtney Before the Kinetograph,5.5


In [40]:
mean_averageRating = dfmergetwo['averageRating'].mean()
print (mean_averageRating)

6.768652980099126


3EME REQUETE

In [54]:
#Compte du nombre de "genre multiple" unique
titlebasics = pd.read_table("./data/title.basics.tsv")
resultat = titlebasics.genres.value_counts()
print (resultat)

Drama                          610744
\N                             501313
Comedy                         463844
Documentary                    311479
Talk-Show                      309046
                                ...  
Action,Documentary,Thriller         1
Adult,Animation,Documentary         1
Crime,Family,News                   1
Crime,Documentary,Musical           1
Animation,Reality-TV,Short          1
Name: genres, Length: 2235, dtype: int64


In [55]:
splittitle = titlebasics["genres"].str.split(",", n = 1, expand = True)
print(titlebasics)


tconst  titleType               primaryTitle  \
0        tt0000001      short                 Carmencita   
1        tt0000002      short     Le clown et ses chiens   
2        tt0000003      short             Pauvre Pierrot   
3        tt0000004      short                Un bon bock   
4        tt0000005      short           Blacksmith Scene   
...            ...        ...                        ...   
6321297  tt9916848  tvEpisode              Episode #3.17   
6321298  tt9916850  tvEpisode              Episode #3.19   
6321299  tt9916852  tvEpisode              Episode #3.20   
6321300  tt9916856      short                   The Wind   
6321301  tt9916880  tvEpisode  Horrid Henry Knows It All   

                     originalTitle  isAdult startYear endYear runtimeMinutes  \
0                       Carmencita        0      1894      \N              1   
1           Le clown et ses chiens        0      1892      \N              5   
2                   Pauvre Pierrot        0      18

In [48]:
titleratings = pd.read_table('./data/title.ratings.tsv')

In [49]:
titleratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1550
1,tt0000002,6.1,186
2,tt0000003,6.5,1207
3,tt0000004,6.2,113
4,tt0000005,6.1,1934
